<a href="https://colab.research.google.com/github/darwinyusef/100BackendTaskingRoadmap/blob/master/ex4/pruebasconSpeetextImageOpenai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata

key = userdata.get('openIAKey')

In [ ]:
from pydantic import BaseModel
from openai import OpenAI

client = OpenAI(api_key=key)  # Reemplaza con tu clave de API

class VocabularyStep(BaseModel):
    word: str
    definition: str
    example_sentence: str
    #image_url: str  # URL de la imagen generada por DALL·E

class VocabularyResponse(BaseModel):
    words: list[VocabularyStep]

# Generar vocabulario
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful language tutor. Provide simple definitions, example sentences, and image prompts for A2 vocabulary words."},
        {"role": "user", "content": "Can you teach me five A2 words related to food?"}
    ],
    response_format=VocabularyResponse,
)

vocabulary_response = completion.choices[0].message.parsed

# Generar imágenes para cada palabra
"""for word_data in vocabulary_response.words:
    image_response = client.images.generate(
        model="dall-e-3",
        prompt=f"An illustration representing the word '{word_data.word}'. {word_data.definition}. Example: {word_data.example_sentence}.",
        size="1024x1024"
    )
    word_data.image_url = image_response.data[0].url  # Asigna la imagen generada"""

vocabulary_response

# Ahora vocabulary_response.words tiene palabras con sus imágenes


In [ ]:
word_data

In [ ]:
!pip install gtts gradio

In [ ]:
import gradio as gr
from gtts import gTTS
import tempfile

# Datos del primer elemento
word_data = {
    "word": "apple",
    "definition": "A round fruit with red, green, or yellow skin and a firm white flesh.",
    "example_sentence": "I eat an apple every morning for breakfast."
}

# Función para generar audio
def generate_audio():
    text = f"Word: {word_data['word']}. Definition: {word_data['definition']}. Example: {word_data['example_sentence']}."
    tts = gTTS(text=text, lang='en', slow=False)

    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)

    return temp_file.name

# Interfaz con Gradio
iface = gr.Interface(
    fn=generate_audio,
    inputs=[],
    outputs=gr.Audio(label="Generated Speech"),
    title="Text to Speech Example",
    description="Click the button to hear the pronunciation of the word, its definition, and an example sentence."
)

iface.launch()
